# First method

In this file we show how to prove that $\varphi_a$ is an isomorphism if and only if $\mathcal{P}_a$ is confluent. We do not use the modules defined to generate properads and ideals, this way we can already compute associativity and coassociativity and save computations.

First we define objects, functions and a class to encode permutations and graphs.

In [1]:
S5 = PermutationGroup(['(1,2)', '(1,3)', '(1, 4)', '(1, 5)'])
S4 = PermutationGroup(['(1,2)', '(1,3)', '(1, 4)'])
S3 = PermutationGroup(['(1,2)', '(1,3)'])
S2 = PermutationGroup(['(1,2)'])

def shift_S3_into_S4(perm, i):
    l=[]
    for j in range(i):
        l.append(j+1)
    for j in range(4-i):
        l.append(perm(j+1)+i)
    return S4(l)

def shift_S4_into_S5(perm, i):
    l=[]
    for j in range(i):
        l.append(j+1)
    for j in range(5-i):
        l.append(perm(j+1)+i)
    return S5(l)

onetwo_S3 = S3.gen(0)
onethree_S3 = S3.gen(1)
onetwo_S2 = S2.gen(0)

permutations_S2 = [S2([1, 2]), S2([2, 1])]
permutations_S3 = [S3([1, 2, 3]), S3([1, 3, 2]), S3([2, 1, 3]), S3([2, 3, 1]), S3([3, 1, 2]), S3([3, 2, 1])]
permutations_S4 = [S4([1, 2, 3, 4]), S4([1, 2, 4, 3]), S4([1, 3, 2, 4]), S4([1, 3, 4, 2]), S4([1, 4, 2, 3]), S4([1, 4, 3, 2]), S4([2, 1, 3, 4]), S4([2, 1, 4, 3]), S4([2, 3, 1, 4]), S4([2, 3, 4, 1]), S4([2, 4, 1, 3]), S4([2, 4, 3, 1]), S4([3, 1, 2, 4]), S4([3, 1, 4, 2]), S4([3, 2, 1, 4]), S4([3, 2, 4, 1]), S4([3, 4, 1, 2]), S4([3, 4, 2, 1]), S4([4, 1, 2, 3]), S4([4, 1, 3, 2]), S4([4, 2, 1, 3]), S4([4, 2, 3, 1]), S4([4, 3, 1, 2]), S4([4, 3, 2, 1])]
permutations_S5 = []
for perm in permutations_S4:
        permutations_S5.append(shift_S4_into_S5(perm, 1)*S5([1, 2, 3, 4, 5]))
for perm in permutations_S4:
        permutations_S5.append(shift_S4_into_S5(perm, 1)*S5([2, 1, 3, 4, 5]))
for perm in permutations_S4:
        permutations_S5.append(shift_S4_into_S5(perm, 1)*S5([3, 1, 2, 4, 5]))
for perm in permutations_S4:
        permutations_S5.append(shift_S4_into_S5(perm, 1)*S5([4, 1, 2, 3, 5]))    
for perm in permutations_S4:
        permutations_S5.append(shift_S4_into_S5(perm, 1)*S5([5, 1, 2, 3, 4]))
        
class Graph:
    
    def __init__(self, outputs, inputs, shape, arity):
        self.outputs = outputs
        self.inputs = inputs
        self.shape = shape
        self.arity = arity
        
    def outputs_image(self):
        s = self.outputs
        return [s(i) for i in range(1, self.arity[0]+1)]
    
    def outputs_preimage(self):
        s = self.outputs.inverse()
        return [s(i) for i in range(1, self.arity[0]+1)]
        
    def inputs_image(self):
        s = self.inputs
        return [s(i) for i in range(1, self.arity[1]+1)]
    
    def inputs_preimage(self):
        s = self.inputs.inverse()
        return [s(i) for i in range(1, self.arity[1]+1)]
    
    def __repr__(self):
        return str(self.outputs_image()) + str(self.shape) + str(self.inputs_preimage()) + " Permutations : " + str(self.outputs) + " " + str(self.inputs)
    
    def __eq__(self, other):
        return self.outputs == other.outputs and self.inputs == other.inputs and self.shape == other.shape and self.arity == other.arity
    
    def copie(self):
        return Graph(self.outputs, self.inputs, self.shape, self.arity)

Then we define this list of generators and polynomial ring.

In [2]:
generators_23 = [0 for i in range(264)] 

permutations_23 = [] 
for i in permutations_S2: 
    for j in permutations_S3:
        permutations_23.append([i,j])
        
for j in range(22): 
    for i in range(12): 
        generators_23[i + j*12] = Graph(permutations_23[i][0], permutations_23[i][1], j, (2, 3))
        
Ra.<a1, a2, a3, a4> = QQ[]

Finally, we construct the matrix

In [3]:
M = [[0 for i in range(180)] for j in range(72)]

#First relation : shape 0 -> a2 shape 5 + a3 shape 2 + a4 (12) * shape 6  + a5 shape 3

relation_init = []
relation_init.append(Graph(S2(), S3(), 0, (2, 3)))
relation_init.append(Graph(S2(), S3(), 5, (2, 3)))
relation_init.append(Graph(S2(), S3(), 2, (2, 3)))
relation_init.append(Graph(S2((1, 2)), S3(), 6, (2, 3)))
relation_init.append(Graph(S2(), S3(), 3, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j][indices[0]] = 1
        M[i*6+j][indices[1]] = -a1
        M[i*6+j][indices[2]] = -a2
        M[i*6+j][indices[3]] = -a3
        M[i*6+j][indices[4]] = -a4
    

#Second relation : shape 2 -> a2 shape 11 + a3 shape 7 + a4 shape 10 * (23) + a5 shape 13 * (23)

relation_init = []
relation_init.append(Graph(S2(), S3(), 2, (2, 3)))
relation_init.append(Graph(S2(), S3(), 11, (2, 3)))
relation_init.append(Graph(S2(), S3(), 7, (2, 3)))
relation_init.append(Graph(S2(), S3((2,3)), 10, (2, 3)))
relation_init.append(Graph(S2(), S3((2,3)), 13, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j+12][indices[0]] = 1
        M[i*6+j+12][indices[1]] = -a1
        M[i*6+j+12][indices[2]] = -a2
        M[i*6+j+12][indices[3]] = -a3
        M[i*6+j+12][indices[4]] = -a4

#Third relation : shape 3 -> a2 shape 9 * (12) + a3 shape 13 * (123) + a4 shape 14 * (13) + a5 (12) * shape 8

relation_init = []
relation_init.append(Graph(S2(), S3(), 3, (2, 3)))
relation_init.append(Graph(S2(), S3((1,2)), 9, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 3, 2)), 13, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 3)), 14, (2, 3)))
relation_init.append(Graph(S2((1, 2)), S3(), 8, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j+24][indices[0]] = 1
        M[i*6+j+24][indices[1]] = -a1
        M[i*6+j+24][indices[2]] = -a2
        M[i*6+j+24][indices[3]] = -a3
        M[i*6+j+24][indices[4]] = -a4
    
#Fourth relation : shape 0 -> a2 shape 1 + a3 shape 7 + a4 shape 4  + a5 (12) * shape 8

relation_init = []
relation_init.append(Graph(S2(), S3(), 0, (2, 3)))
relation_init.append(Graph(S2(), S3(), 1, (2, 3)))
relation_init.append(Graph(S2(), S3(), 7, (2, 3)))
relation_init.append(Graph(S2(), S3(), 4, (2, 3)))
relation_init.append(Graph(S2((1, 2)), S3(), 8, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j+36][indices[0]] = 1
        M[i*6+j+36][indices[1]] = -a1
        M[i*6+j+36][indices[2]] = -a2
        M[i*6+j+36][indices[3]] = -a3
        M[i*6+j+36][indices[4]] = -a4

#Fifth relation : shape 1 -> a2 shape 5 + a3 shape 11 + a4 shape 12 * (12)  + a5 shape 9 * (12)

relation_init = []
relation_init.append(Graph(S2(), S3(), 1, (2, 3)))
relation_init.append(Graph(S2(), S3(), 5, (2, 3)))
relation_init.append(Graph(S2(), S3(), 11, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 2)), 12, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 2)), 9, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j+48][indices[0]] = 1
        M[i*6+j+48][indices[1]] = -a1
        M[i*6+j+48][indices[2]] = -a2
        M[i*6+j+48][indices[3]] = -a3
        M[i*6+j+48][indices[4]] = -a4

#Sixth relation : shape 4 -> a2 shape 12 * (132) + a3 shape 10 * (23) + a4 (12) * shape 6  + a5 shape 14 * (13)

relation_init = []
relation_init.append(Graph(S2(), S3(), 4, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 2, 3)), 12, (2, 3)))
relation_init.append(Graph(S2(), S3((2, 3)), 10, (2, 3)))
relation_init.append(Graph(S2((1, 2)), S3(), 6, (2, 3)))
relation_init.append(Graph(S2(), S3((1, 3)), 14, (2, 3)))

for i in range(2):
    for j in range(6):
        perm_inputs = permutations_S3[j]
        perm_outputs = permutations_S2[i]
        relation = []
        for g in relation_init:
            gtemp = g.copie()
            gtemp.inputs = perm_inputs * gtemp.inputs 
            gtemp.outputs = perm_outputs * gtemp.outputs
            relation.append(gtemp)
        indices = [generators_23.index(g) for g in relation]
        M[i*6+j+60][indices[0]] = 1
        M[i*6+j+60][indices[1]] = -a1
        M[i*6+j+60][indices[2]] = -a2
        M[i*6+j+60][indices[3]] = -a3
        M[i*6+j+60][indices[4]] = -a4


Here we can print the matrix row by row to see what operations we have to do. "xen" means that the coefficient in column n is x.

In [4]:
def print_matrix(mat, i):
    matrix = mat[i]
    message = ["Row " + str(i) + " : "]
    for j in range(len(matrix)):
        if matrix[j] != 0:
            message.append(str(matrix[j]) + "e" + str(j))
    print(message)

for i in range(72):
    print_matrix(M,i)

['Row 0 : ', '1e0', '-a2e24', '-a4e36', '-a1e60', '-a3e78']
['Row 1 : ', '1e1', '-a2e25', '-a4e37', '-a1e61', '-a3e79']
['Row 2 : ', '1e2', '-a2e26', '-a4e38', '-a1e62', '-a3e80']
['Row 3 : ', '1e3', '-a2e27', '-a4e39', '-a1e63', '-a3e81']
['Row 4 : ', '1e4', '-a2e28', '-a4e40', '-a1e64', '-a3e82']
['Row 5 : ', '1e5', '-a2e29', '-a4e41', '-a1e65', '-a3e83']
['Row 6 : ', '1e6', '-a2e30', '-a4e42', '-a1e66', '-a3e72']
['Row 7 : ', '1e7', '-a2e31', '-a4e43', '-a1e67', '-a3e73']
['Row 8 : ', '1e8', '-a2e32', '-a4e44', '-a1e68', '-a3e74']
['Row 9 : ', '1e9', '-a2e33', '-a4e45', '-a1e69', '-a3e75']
['Row 10 : ', '1e10', '-a2e34', '-a4e46', '-a1e70', '-a3e76']
['Row 11 : ', '1e11', '-a2e35', '-a4e47', '-a1e71', '-a3e77']
['Row 12 : ', '1e24', '-a2e84', '-a3e121', '-a1e132', '-a4e157']
['Row 13 : ', '1e25', '-a2e85', '-a3e120', '-a1e133', '-a4e156']
['Row 14 : ', '1e26', '-a2e86', '-a3e124', '-a1e134', '-a4e160']
['Row 15 : ', '1e27', '-a2e87', '-a3e125', '-a1e135', '-a4e161']
['Row 16 : ', '1

Now that we know what operations to do, we do them and print the new matrix

In [5]:
matrix = matrix(M)
print(matrix.rank())
for i in range(12):
    matrix.swap_rows(i + 12, i + 48)
for i in range(12):
    matrix.swap_rows(i + 24, i + 48)
for i in range(12):
    matrix.swap_rows(i + 36, i + 48)
for i in range(12):
    matrix.swap_rows(i + 60, i + 48)
for i in range(12):
    matrix = matrix.with_added_multiple_of_row(60 + i, i, -1)
for i in range(12):
    matrix = matrix.with_added_multiple_of_row(60 + i, i + 12, a1)
for i in range(12):
    matrix = matrix.with_added_multiple_of_row(60 + i, i + 24, -a2)
for i in range(12):
    matrix = matrix.with_added_multiple_of_row(60 + i, i + 36, -a4)
for i in range(12):
    matrix = matrix.with_added_multiple_of_row(60 + i, i + 48, a3)
    
for i in range(72):
    print_matrix(matrix, i)

72
['Row 0 : ', '1e0', '-a2e24', '-a4e36', '-a1e60', '-a3e78']
['Row 1 : ', '1e1', '-a2e25', '-a4e37', '-a1e61', '-a3e79']
['Row 2 : ', '1e2', '-a2e26', '-a4e38', '-a1e62', '-a3e80']
['Row 3 : ', '1e3', '-a2e27', '-a4e39', '-a1e63', '-a3e81']
['Row 4 : ', '1e4', '-a2e28', '-a4e40', '-a1e64', '-a3e82']
['Row 5 : ', '1e5', '-a2e29', '-a4e41', '-a1e65', '-a3e83']
['Row 6 : ', '1e6', '-a2e30', '-a4e42', '-a1e66', '-a3e72']
['Row 7 : ', '1e7', '-a2e31', '-a4e43', '-a1e67', '-a3e73']
['Row 8 : ', '1e8', '-a2e32', '-a4e44', '-a1e68', '-a3e74']
['Row 9 : ', '1e9', '-a2e33', '-a4e45', '-a1e69', '-a3e75']
['Row 10 : ', '1e10', '-a2e34', '-a4e46', '-a1e70', '-a3e76']
['Row 11 : ', '1e11', '-a2e35', '-a4e47', '-a1e71', '-a3e77']
['Row 12 : ', '1e12', '-a1e60', '-a4e110', '-a2e132', '-a3e146']
['Row 13 : ', '1e13', '-a1e61', '-a4e111', '-a2e133', '-a3e147']
['Row 14 : ', '1e14', '-a1e62', '-a4e108', '-a2e134', '-a3e144']
['Row 15 : ', '1e15', '-a1e63', '-a4e109', '-a2e135', '-a3e145']
['Row 16 : ',